# Designing an Intelligent Agent for the Wumpus World - Part 1

The Wumpus World is a classic problem in artificial intelligence that involves designing an agent to navigate a grid-based environment filled with hazards, such as pits and a fearsome Wumpus, while searching for gold. The agent must operate under uncertainty, leveraging logical reasoning and perception to make decisions in a partially observable world.

This project focuses on implementing an intelligent Wumpus World agent capable of:

- Perceiving environmental cues such as breezes (near pits) and stenches (near the Wumpus).
- Applying logical inference to deduce the safe path to the gold.
- Balancing exploration and caution to avoid fatal encounters.

The project highlights key AI concepts, including decision-making under uncertainty, logical reasoning, and search algorithms, providing a practical application of intelligent agent design in a challenging and dynamic environment.

In [1]:
from typing import List
from enum import Enum, auto
import random

In [2]:
class Percept():
    time_step: int
    bump: bool
    breeze: bool
    stench: bool
    scream: bool
    glitter: bool
    reward: int
    done: bool
        
    def __init__(self, time_step: int, bump: bool, breeze: bool, stench: bool, scream: bool, glitter: bool, reward: bool, done: bool):
         self.time_step = time_step
         self.bump = bump
         self.breeze = breeze
         self.stench = stench
         self.scream = scream
         self.glitter = glitter
         self.reward = reward
         self.done = done

    def __str__(self):
        return f"\ntime_step: {self.time_step}\nbump: {self.bump}\nstench: {self.stench}\nbreeze: {self.breeze}\nscream: {self.scream}\nglitter: {self.glitter}\nreward: {self.reward}"

In [3]:
class Action(Enum):
    LEFT = 0
    RIGHT = 1
    FORWARD = 2
    GRAB = 3
    SHOOT = 4
    CLIMB = 5

In [4]:
class Orientation(Enum):
    E = 0
    S = 1
    W = 2
    N = 3

    def symbol(self) -> str:
        if self == Orientation.E:
            return ">"
        elif self == Orientation.S:
            return "v"
        elif self == Orientation.W:
            return "<"
        elif self == Orientation.N:
            return "^"
        else:
            return "0"

    def turn_right(self) -> 'Orientation':
        current_index = self.value
        next_index = (current_index + 1) % len(Orientation)
        return Orientation(next_index)
        
    def turn_left(self) -> 'Orientation':
        current_index = self.value
        prev_index = (current_index - 1) % len(Orientation)
        return Orientation(prev_index)

In [5]:
class Verbosity(Enum):
    NONE = 0
    LOGS = 1
    ALL = 2

class NaiveAgent:
    verbosity: Verbosity
    
    def __init__(self, verbosity):
        self.verbosity = verbosity
    
    def choose_action(self):
        return random.choice(list(Action))
    
    def run(self):
        env = Environment()
        cumulative_reward = 0
        percept = env.init(0.2, False)
        while not percept.done:
            if self.verbosity == Verbosity.ALL:
                env.visualize()
            if self.verbosity != Verbosity.NONE:
                print('Percept:', percept)
            action = self.choose_action()
            if self.verbosity != Verbosity.NONE:
                print()
                print('Action:', action)
                print()
            percept = env.step(action)
            cumulative_reward += percept.reward
        if self.verbosity == Verbosity.ALL:
            env.visualize()
        if self.verbosity != Verbosity.NONE:
            print('Percept:', percept)
        print('Cumulative reward:', cumulative_reward)

In [6]:
class Location:
    x: int
    y: int
        
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f'({self.x}, {self.y})'
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    
    def is_left_of(self, location: 'Location')->bool:
        return self.x == location.x-1 and self.y == location.y
        
   
    def is_right_of(self, location: 'Location')->bool:
        return self.x == location.x+1 and self.y == location.y
        
    
    def is_above(self, location: 'Location')->bool:
        return self.x == location.x and self.y == location.y+1
        
    
    def is_below(self, location: 'Location')->bool:
        return self.x == location.x and self.y == location.y-1
        
    
    def neighbours(self)->List['Location']:
        return (Location(self.x-1, self.y), Location(self.x+1, self.y), Location(self.x, self.y+1), Location(self.x, self.y-1))
    
    
    # return True if location given is self's location
    def is_location(self, location: 'Location')->bool:
        return self.x == location.x and self.y == location.y
    
    
    def at_left_edge(self) -> bool:
        return self.x == 1
    
    
    def at_right_edge(self) -> bool:
        # if not 4x4, we need to specify the width and height.
        return self.x == 4
    
    
    def at_top_edge(self) -> bool:
        # if not 4x4, we need to specify the width and height.
        return self.y == 4  
    
   
    def at_bottom_edge(self) -> bool:
        return self.y == 1  
    
    
    def forward(self, orientation) -> bool:
        if orientation == Orientation.N:
            self.y += 1
        elif orientation == Orientation.W:
            self.x -= 1
        elif orientation == Orientation.S:
            self.y -= 1
        elif orientation == Orientation.E:
            self.x += 1
        
        if self.x < 1:
            self.x = 1
            return True
        elif self.x > 4:
            self.x = 4
            return True
        
        if self.y < 1:
            self.y = 1
            return True
        elif self.y > 4:
            self.y = 4
            return True
        return False

    def set_to(self, location: 'Location'):
        self.x = location.x
        self.y = location.y
        
    
    @staticmethod
    def from_linear(n: int) -> 'Location':
        pass
        # convert an index from 0 to 15 to a location
    
    
    def to_linear(self)->int:
        pass
        # convert self to an index from 0 to 15    
    
    
    @staticmethod
    def random() -> 'Location':
        x = random.randint(1, 4)
        y = random.randint(1, 4)
        return Location(x, y)

In [7]:
class Environment:
    wumpus_location: Location
    wumpus_alive: bool
    agent_location: Location
    agent_orientation: Orientation
    agent_has_arrow: bool
    agent_has_gold: bool
    game_over: bool
    gold_location: Location
    pit_locations: List[Location]
    time_step: int
    allow_climb_without_gold: bool
        
    def init(self, pit_prob: float, allow_climb_without_gold: bool):
        self.make_wumpus()
        self.make_gold()
        self.pit_locations = []
        self.make_pits(pit_prob)
        
        self.agent_location = Location(1,1)
        self.agent_orientation = Orientation.E
        self.agent_has_arrow = True
        self.agent_has_gold = False
        
        self.game_over = False
        self.time_step = 0
        self.allow_climb_without_gold = allow_climb_without_gold
        return Percept(self.time_step, False, self.is_breeze(), self.is_stench(), False, self.is_glitter(), 0, self.game_over)
         
        
    def get_random_location(self):
        temp_loc = None
        while True:
            temp_loc = Location.random()
            if not (temp_loc.x == 1 and temp_loc.y == 1):
                break
        return temp_loc
            
    def make_wumpus(self):
        self.wumpus_location = self.get_random_location()
        self.wumpus_alive = True
 
    def make_gold(self):
        self.gold_location = self.get_random_location()

    def make_pits(self, pit_prob: float):
        for y in range(1,5):
            for x in range(1,5):
                if x == 1 and y == 1:
                    pass
                elif random.random() <= pit_prob:
                    self.pit_locations.append(Location(x, y))
 
    def is_pit_at(self, location: Location) -> bool:
        if location in self.pit_locations:
            return True
        return False
        
    def is_pit_adjacent_to_agent(self) -> bool:
        for neighbor in self.agent_location.neighbours():
            if neighbor in self.pit_locations:
                return True
        return False

    def is_wumpus_adjacent_to_agent(self) -> bool:
        if self.wumpus_location in self.agent_location.neighbours():
            return True
        return False
        
    def is_agent_at_hazard(self)->bool:
        if self.agent_location is self.wumpus_location or\
            self.agent_location in self.pit_locations:
                return True
        return False

    def is_wumpus_at(self, location: Location) -> bool:
        return self.wumpus_location == location
        
    def is_agent_at(self, location: Location) -> bool:
        return self.agent_location == location 
    
    def is_gold_at(self, location: Location) -> bool:
        return self.gold_location == location
        
    def is_glitter(self) -> bool:
        return self.agent_location == self.gold_location
        
    def is_breeze(self) -> bool:
        if self.agent_location in self.pit_locations:
            return True
        for neighbor in self.agent_location.neighbours():
            if neighbor in self.pit_locations:
                return True
        return False

    def is_stench(self) -> bool:
        if self.agent_location == self.wumpus_location:
            return True
        for neighbor in self.agent_location.neighbours():
            if neighbor == self.wumpus_location:
                return True
        return False

    def wumpus_in_line_of_fire(self) -> bool:
        line_of_fire = []
        if self.agent_orientation == Orientation.N:
            if self.agent_location.y < 4:
                for y in range(self.agent_location.y, 5):
                    line_of_fire.append(Location(self.agent_location.x, y))
        if self.agent_orientation == Orientation.S:
            if self.agent_location.y > 1:
                for y in range(self.agent_location.y, -1, 0):
                    line_of_fire.append(Location(self.agent_location.x, y))
        if self.agent_orientation == Orientation.E:
            if self.agent_location.x < 4:
                for x in range(self.agent_location.x, 5):
                    line_of_fire.append(Location(x, self.agent_location.y))
        if self.agent_orientation == Orientation.W:
            if self.agent_location.x > 1:
                for x in range(self.agent_location.x, -1, 0):
                    line_of_fire.append(Location(x, self.agent_location.y))
                    
        if self.wumpus_location in line_of_fire:
            return True
        return False

    def kill_attempt(self) -> bool:
        if self.wumpus_in_line_of_fire() and self.wumpus_alive:
            self.wumpus_alive = False
            return True
        return False
        
    def step(self, action: Action) -> Percept:
        bump = False
        scream = False
        reward = -1
        if action == Action.FORWARD:
            bump = self.agent_location.forward(self.agent_orientation)
            if self.agent_has_gold:
                self.gold_location = self.agent_location
            if self.is_wumpus_at(self.agent_location) and self.wumpus_alive:
                reward -= 1000
                self.game_over = True
            if self.is_pit_at(self.agent_location) and not self.game_over:
                reward -= 1000
                self.game_over = True
        if action == Action.SHOOT and self.agent_has_arrow:
            reward -= 10
            scream = self.kill_attempt()
            self.agent_has_arrow = False
        if action == Action.CLIMB:
            if self.agent_location == Location(1,1):
                if self.allow_climb_without_gold:
                    self.game_over = True
                if self.agent_has_gold:
                    self.game_over = True
                    reward += 1000
        if action == Action.LEFT:
            self.agent_orientation = self.agent_orientation.turn_left()
        if action == Action.RIGHT:
            self.agent_orientation = self.agent_orientation.turn_right()
        if action == Action.GRAB and self.is_gold_at(self.agent_location):
            self.agent_has_gold = True
        
        self.time_step += 1
        return Percept(self.time_step, bump, self.is_breeze(), self.is_stench(), scream, self.is_glitter(), reward, self.game_over)
    
    # Visualize the game state
    def visualize(self):
        for y in range(3, -1, -1):
            line = '|'
            for x in range(0, 4):
                loc = Location(x+1, y+1)
                cell_symbols = [' ', ' ', ' ', ' ']
                if self.is_agent_at(loc): cell_symbols[0] = self.agent_orientation.symbol()
                if self.is_pit_at(loc): cell_symbols[1] = 'P'
                if self.is_wumpus_at(loc):
                    if self.wumpus_alive:
                        cell_symbols[2] = 'W'
                    else:
                        cell_symbols[2] = 'w'
                if self.is_gold_at(loc): cell_symbols[3] = 'G'
                for char in cell_symbols: line += char
                line += '|'
            print(line)

In [8]:
# Verbosity has 3 levels, NONE will show only the cumulative score, LOGS will show the percept logs
# ALL will show both visualization and logs together.
NaiveAgent(Verbosity.ALL).run()

|   G|    |    |    |
|    |    |  W |    |
| P  |    |    |    |
|>   |    |    | P  |
Percept: 
time_step: 0
bump: False
stench: False
breeze: True
scream: False
glitter: False
reward: 0

Action: Action.SHOOT

|   G|    |    |    |
|    |    |  W |    |
| P  |    |    |    |
|>   |    |    | P  |
Percept: 
time_step: 1
bump: False
stench: False
breeze: True
scream: False
glitter: False
reward: -11

Action: Action.GRAB

|   G|    |    |    |
|    |    |  W |    |
| P  |    |    |    |
|>   |    |    | P  |
Percept: 
time_step: 2
bump: False
stench: False
breeze: True
scream: False
glitter: False
reward: -1

Action: Action.LEFT

|   G|    |    |    |
|    |    |  W |    |
| P  |    |    |    |
|^   |    |    | P  |
Percept: 
time_step: 3
bump: False
stench: False
breeze: True
scream: False
glitter: False
reward: -1

Action: Action.RIGHT

|   G|    |    |    |
|    |    |  W |    |
| P  |    |    |    |
|>   |    |    | P  |
Percept: 
time_step: 4
bump: False
stench: False
breeze: True
